In [1]:
import numpy as np
import pandas as pd
import os
import cv2
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.applications import MobileNet
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

/home/aman/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aman/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aman/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aman/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWa

In [2]:
df = pd.read_csv("dataset/train.csv")
df.head()

,Image,Class
0,image7042.jpg,Food
1,image3327.jpg,misc
2,image10335.jpg,Attire
3,image8019.jpg,Food
4,image2128.jpg,Attire


In [3]:
trainPath = "dataset/Train Images/"
Images = os.listdir(trainPath)

In [4]:
len(Images)-len(df)

0

In [5]:
print(df.isna().sum())

Image    0
Class    0
dtype: int64


In [6]:
df.groupby(by = 'Class', axis=0).count()

,Image
Class,
Attire,1691
Decorationandsignage,743
Food,2278
misc,1271


In [7]:
temp = []

for i in range(len(df)):
    img = cv2.imread(trainPath + Images[i])
    img = np.resize(img, (60, 80, 3))
    img = img.astype('float')
    img = img/255.0
    
    temp.append({"Image" : Images[i], "imageData" : img})

temp = pd.DataFrame(temp)
temp.head()

,Image,imageData
0,image5402.jpg,"[[[1.0, 1.0, 1.0], [1.0, 1.0, 1.0], [1.0, 1.0,..."
1,image9707.jpg,"[[[0.7333333333333333, 0.7529411764705882, 0.7..."
2,image10325.jpg,"[[[0.09411764705882353, 0.043137254901960784, ..."
3,image3662.jpg,"[[[0.3764705882352941, 0.592156862745098, 0.10..."
4,image2709.jpg,"[[[0.2549019607843137, 0.2627450980392157, 0.2..."


In [8]:
df = pd.merge(df, temp, on = "Image")
df.head()

,Image,Class,imageData
0,image7042.jpg,Food,"[[[0.17647058823529413, 0.17647058823529413, 0..."
1,image3327.jpg,misc,"[[[0.38823529411764707, 0.47058823529411764, 0..."
2,image10335.jpg,Attire,"[[[0.027450980392156862, 0.058823529411764705,..."
3,image8019.jpg,Food,"[[[0.10588235294117647, 0.16862745098039217, 0..."
4,image2128.jpg,Attire,"[[[0.6627450980392157, 0.6549019607843137, 0.6..."


In [9]:
le = LabelEncoder()
df["Class"] = le.fit_transform(df.Class)
df.head()

,Image,Class,imageData
0,image7042.jpg,2,"[[[0.17647058823529413, 0.17647058823529413, 0..."
1,image3327.jpg,3,"[[[0.38823529411764707, 0.47058823529411764, 0..."
2,image10335.jpg,0,"[[[0.027450980392156862, 0.058823529411764705,..."
3,image8019.jpg,2,"[[[0.10588235294117647, 0.16862745098039217, 0..."
4,image2128.jpg,0,"[[[0.6627450980392157, 0.6549019607843137, 0.6..."


In [10]:
df.Class.unique()

array([2, 3, 0, 1])

In [11]:
le.inverse_transform(df.Class)

array(['Food', 'misc', 'Attire', ..., 'Food', 'Food', 'Attire'],
      dtype=object)

In [12]:
x = []
y = []

for i in range(len(df)):
    x.append(df.imageData[i])
    y.append(df.Class[i])
    
x = np.array(x)
y = np.array(y)
y = to_categorical(y)

In [13]:
x.shape

(5983, 60, 80, 3)

In [14]:
y.shape

(5983, 4)

In [15]:
def buildModel():
    baseModel = MobileNet(input_shape=(60, 80, 3), weights='imagenet',include_top=False)
    model = Sequential()
    model.add(baseModel)
    model.add(GlobalAveragePooling2D())
    
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    
    model.add(Dense(4, activation='softmax'))
    
    baseModel.trainable=False
    
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [16]:
model = buildModel()
model.summary()

/home/aman/.local/lib/python3.7/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 1, 2, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
Total params: 3,507,972
Trainable params: 279,108
Non-trainable params: 3,228,864
_________________________________________________________________


In [17]:
model.fit(x, y, epochs=2, verbose=1, validation_split=0.2)


Train on 4786 samples, validate on 1197 samples
Epoch 1/2
4786/4786 [==============================] - 28s 6ms/step - loss: 1.1456 - accuracy: 0.5104 - val_loss: 1.1679 - val_accuracy: 0.5163
Epoch 2/2
4786/4786 [==============================] - 27s 6ms/step - loss: 0.9226 - accuracy: 0.6166 - val_loss: 1.1499 - val_accuracy: 0.5272


In [18]:
loss, accuracy = model.evaluate(x, y, verbose=1)
print(str(loss) + " : " + str(accuracy))

5983/5983 [==============================] - 36s 6ms/step
1.1599216740326241 : 0.5191375613212585


In [16]:
np.argmax(model.predict(x), axis=1)

array([2, 3, 0, ..., 2, 2, 0])

In [17]:
np.argmax(y, axis=1)

array([2, 3, 0, ..., 2, 2, 0])

In [18]:
df = pd.read_csv("dataset/test.csv")
df.head()

,Image
0,image6245.jpg
1,image10409.jpg
2,image8692.jpg
3,image10517.jpg
4,image2580.jpg


In [19]:
testPath = "dataset/Test Images/"
Images = os.listdir(testPath)
len(Images)

3219

In [20]:
df.isna().sum()

Image    0
dtype: int64

In [21]:
temp = []

for i in range(len(df)):
    img = cv2.imread(testPath + Images[i])
    img = np.resize(img, (60, 60, 3))
    img = img.astype('float')
    img = img/255.0
    
    temp.append({"Image" : Images[i], "imageData" : img})

temp = pd.DataFrame(temp)
temp.head()

,Image,imageData
0,image5761.jpg,"[[[0.043137254901960784, 0.11372549019607843, ..."
1,image7487.jpg,"[[[0.2980392156862745, 0.34509803921568627, 0...."
2,image4596.jpg,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
3,image8532.jpg,"[[[0.9098039215686274, 0.6901960784313725, 0.3..."
4,image1449.jpg,"[[[0.5529411764705883, 0.596078431372549, 0.67..."


In [22]:
df = pd.merge(df, temp, on = "Image")
df.head()

,Image,imageData
0,image6245.jpg,"[[[0.9921568627450981, 0.984313725490196, 0.98..."
1,image10409.jpg,"[[[0.0, 0.0, 0.011764705882352941], [0.0, 0.0,..."
2,image8692.jpg,"[[[0.3333333333333333, 0.36470588235294116, 0...."
3,image10517.jpg,"[[[0.4, 0.17647058823529413, 0.207843137254901..."
4,image2580.jpg,"[[[0.34901960784313724, 0.34901960784313724, 0..."


In [23]:
x = []

for i in range(len(df)):
    x.append(df.imageData[i])
    
x = np.array(x)

In [24]:
predict = np.argmax(model.predict(x), axis=1)

In [25]:
predict

array([2, 0, 2, ..., 0, 2, 3])

In [26]:
predict = le.inverse_transform(predict)
predict

array(['Food', 'Attire', 'Food', ..., 'Attire', 'Food', 'misc'],
      dtype=object)

In [27]:
df["Class"] = predict

In [28]:
df.head()

,Image,imageData,Class
0,image6245.jpg,"[[[0.9921568627450981, 0.984313725490196, 0.98...",Food
1,image10409.jpg,"[[[0.0, 0.0, 0.011764705882352941], [0.0, 0.0,...",Attire
2,image8692.jpg,"[[[0.3333333333333333, 0.36470588235294116, 0....",Food
3,image10517.jpg,"[[[0.4, 0.17647058823529413, 0.207843137254901...",misc
4,image2580.jpg,"[[[0.34901960784313724, 0.34901960784313724, 0...",misc


In [29]:
df = df.drop(['imageData'], axis=1)

In [30]:
df.head()

,Image,Class
0,image6245.jpg,Food
1,image10409.jpg,Attire
2,image8692.jpg,Food
3,image10517.jpg,misc
4,image2580.jpg,misc


In [31]:
df.to_csv("output/output.csv")